In [74]:
from PIL import Image
import pytesseract
import argparse
import cv2
import os
import re
import io
import json
import datetime

from dateutil.relativedelta import relativedelta

In [75]:
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

In [76]:
def common(path, blur=True):
    # load the example image and convert it to grayscale
    image = cv2.imread(path)
    imgres = cv2.resize(image, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)
    gray = cv2.cvtColor(imgres, cv2.COLOR_BGR2GRAY)
    
    if blur:
        gray = cv2.GaussianBlur(gray, (5,5), 0)
    
    # write the grayscale image to disk as a temporary file so we can apply OCR to it
    filename = "{}.png".format(os.getpid())
    cv2.imwrite(filename, gray)

    # write the grayscale image to disk as a temporary file so we can apply OCR to it
    filename = "{}.png".format(os.getpid())
    cv2.imwrite(filename, gray)

    # load the image as a PIL/Pillow image, apply OCR, and then delete the temporary file
    text = pytesseract.image_to_string(Image.open(filename), lang = 'eng')
    os.remove(filename)
    #print(text)

    return text

In [102]:
def aadhaarfront():

    txt = common("C:/Users/Pradyumna/Downloads/a5.jpeg", True)
    
    text = []
    data = {
        'firstName': None,
        'lastName': None,
        'age': 18,
        'gender': None,
        'cellPhone': None,
        'documentNumber': None
    }

    c = 0
    pos = 0
    mobpos = 0

    # Splitting lines
    lines = txt.split('\n')
    
    for lin in lines:
        s = lin.strip()
        s = s.replace('\n','')
        if s:
            c = c + 1
            s = s.rstrip()
            s = s.lstrip()
            text.append(s)
            if re.match(r".*male|.*MALE", s):
                pos = c
            if re.match(r".*mobile|.*obile|.*Mobile", s):
                mobpos = c
        
            
    print(txt)

    try:
        # Cleaning Name
        name = text[pos-3].rstrip()
        name = name.lstrip()
        name = re.sub('[^a-zA-Z]+', ' ', name)
        nmlt = name.split(" ")
        data['firstName'] = " ".join(nmlt[:len(nmlt)-1])
        data['lastName'] = nmlt[-1]

        # Cleaning YOB
        yob = text[pos-2].replace("/", " ")
        yob = re.sub('[^0-9]+', ' ', yob)
        yob = yob.split(" ")[-1]
        yob = yob.rstrip()
        yob = int(yob.lstrip())
        data['age'] = datetime.datetime.now().year - yob

        # Cleaning Gender
        data['gender'] = text[pos-1].split(" ", 2)[-1]
        data['gender'] = data['gender'].replace('/', '')
        data['gender'] = data['gender'].replace('(', '')
        data['gender'] = data['gender'].rstrip()
        data['gender'] = data['gender'].lstrip().lower()
        
        # Cleaning Mobile Number
        if mobpos > 0:
            data['cellPhone'] = re.sub('[^0-9]+', '', text[mobpos-1])
            data['cellPhone'] = data['cellPhone'].rstrip()
            data['cellPhone'] = int(data['cellPhone'].lstrip())

        # Cleaning Aadhar Number
        for i in range(pos, len(text)):
            if re.match(r'[0-9]', text[i]):
                data['documentNumber'] = re.sub('[^0-9]+', '', text[i])
                data['documentNumber'] = data['documentNumber'].rstrip()
                data['documentNumber'] = int(data['documentNumber'].lstrip())
    except:
        pass

    print(data)

In [116]:
def aadhaarback():

    txt = common("C:/Users/Pradyumna/Downloads/ab1.jpeg", True)
    
    text = []
    data = {
        'address1': None,
        'address2': None,
        'zip': 18,
        'state': None,
        'city': None,
    }

    # Splitting lines
    lines = txt.split('\n')
    
    for lin in lines:
        s = lin.strip()
        s = s.replace('\n','')
        if s:
            s = s.rstrip()
            s = s.lstrip()
            text.append(s)
            
            try:
                if re.search(r"\d{6}", s):
                    data['zip'] = re.search(r'\d{6}', s).group()
                    data['state'] = s.split(" ")[-2].replace(',', '')
            except:
                pass
            
    print(txt)
    
    print(data)    

In [108]:
def driverfront():
    txt = common("C:/Users/Pradyumna/Downloads/DL11.jpg", True)
    
    text = []
    data = {
        'firstName': None,
        'lastName': None,
        'age': 18,
        'gender': None,
        'cellPhone': None,
        'documentNumber': None
    }
    
    c = 0
    print(txt)
    
    #Splitting lines
    lines = txt.split('\n')
    
    for lin in lines:
        c = c + 1
        s = lin.strip()
        s = s.replace('\n','')
        if s:
            s = s.rstrip()
            s = s.lstrip()
            text.append(s)

            try:
                if re.match(r".*Name|.*name|.*NAME", s):           
                    name = re.sub('[^a-zA-Z]+', ' ', s)
                    name = name.replace('Name', '')
                    name = name.replace('name', '')
                    name = name.replace('NAME', '')
                    name = name.replace(':', '')
                    name = name.rstrip()
                    name = name.lstrip()
                    nmlt = name.split(" ")
                    data['firstName'] = " ".join(nmlt[:len(nmlt)-1])
                    data['lastName'] = nmlt[-1]
                if re.search(r"[a-zA-Z][a-zA-Z]-\d{13}", s):
                    data['documentNumber'] = re.search(r'[a-zA-Z][a-zA-Z]-\d{13}', s)
                    data['documentNumber'] = data['documentNumber'].group().replace('-', '')
                    if not data['firstName']:
                        name = lines[c]           
                        name = re.sub('[^a-zA-Z]+', ' ', name)
                        name = name.rstrip()
                        name = name.lstrip()
                        nmlt = name.split(" ")
                        data['firstName'] = " ".join(nmlt[:len(nmlt)-1])
                        data['lastName'] = nmlt[-1]
                if re.search(r"[a-zA-Z][a-zA-Z]\d{2} \d{11}", s):
                    data['documentNumber'] = re.search(r'[a-zA-Z][a-zA-Z]\d{2} \d{11}', s)
                    data['documentNumber'] = data['documentNumber'].group().replace(' ', '')
                    if not data['firstName']:
                        name = lines[c]           
                        name = re.sub('[^a-zA-Z]+', ' ', name)
                        name = name.rstrip()
                        name = name.lstrip()
                        nmlt = name.split(" ")
                        data['firstName'] = " ".join(nmlt[:len(nmlt)-1])
                        data['lastName'] = nmlt[-1]
                if re.match(r".*DOB|.*dob|.*Dob", s):         
                    yob = re.sub('[^0-9]+', ' ', s)
                    yob = re.search(r'\d\d\d\d', yob)
                    data['age'] = datetime.datetime.now().year - int(yob.group())
            except:
                pass

    print(data)

In [103]:
aadhaarfront()

gaa sal
Suman Tandon

DOB: 10-05-1956
Gender:Female

 

{'firstName': 'Suman', 'lastName': 'Tandon', 'age': 64, 'gender': 'gender:female', 'cellPhone': None, 'documentNumber': None}


In [109]:
driverfront()

‘1 10/06/2014

 
  
  
  
   
 
  
  
  

 

DLNo -TN39 20140004724

Name LOGANATHAN S$
SD W of  SIVASAMY

Address
26, SENGALIPALAYAM
KUNNATHUR PO, PERUNDURAI
-ERODE

Temp.Add
SARSWATHI DS
DOB. 25/03/1985

  

{'firstName': 'LOGANATHAN', 'lastName': 'S', 'age': 35, 'gender': None, 'cellPhone': None, 'documentNumber': 'TN3920140004724'}


In [117]:
aadhaarback()

A» ade ase gear wrftraxor ai
Za Unique Identification Authority of India
Address:

Uo:
dara: sr. got faent erat, 441, D/O: Dr. Brij Bihari Sharma, 441,
fase aproiter, eRean, @Raan, vikas colony, Hardwar, Haridwar,

SeaRMGS, 249401 Haridwar, Uttarakhand, 249401

|

3283 4137 6801 |
= be |

yA
1800 300 1 947 help @uidai.gov.in www.uidai.gov.in

{'address1': None, 'address2': None, 'zip': '249401', 'state': 'Uttarakhand', 'city': None}
